In [ ]:
!pip install ../input/lama-whl/efficientnet_pytorch-0.7.0/dist/efficientnet_pytorch-0.7.0.tar ../input/lama-whl/log_calls-0.3.2/log_calls-0.3.2/ ../input/lama-whl/sphinxcontrib_devhelp-1.0.2-py2.py3-none-any.whl ../input/lama-whl/sphinxcontrib_htmlhelp-1.0.3-py2.py3-none-any.whl ../input/lama-whl/sphinxcontrib_jsmath-1.0.1-py2.py3-none-any.whl ../input/lama-whl/sphinxcontrib_qthelp-1.0.3-py2.py3-none-any.whl ../input/lama-whl/sphinxcontrib_applehelp-1.0.2-py2.py3-none-any.whl ../input/lama-whl/sphinxcontrib_serializinghtml-1.1.4-py2.py3-none-any.whl ../input/lama-whl/importlib_metadata-1.7.0-py2.py3-none-any.whl ../input/lama-whl/poetry_core-1.0.3-py2.py3-none-any.whl ../input/lama-whl/imagesize-1.2.0-py2.py3-none-any.whl ../input/lama-whl/docutils-0.16-py2.py3-none-any.whl ../input/lama-whl/alabaster-0.7.12-py2.py3-none-any.whl ../input/lama-whl/snowballstemmer-2.1.0-py2.py3-none-any.whl ../input/lama-whl/Sphinx-3.5.4-py3-none-any.whl ../input/lama-whl/sphinx_autodoc_typehints-1.11.1-py3-none-any.whl ../input/lama-whl/nbsphinx-0.8.0-py3-none-any.whl ../input/lama-whl/nbsphinx_link-1.3.0-py2.py3-none-any.whl ../input/lama-whl/cssselect-1.1.0-py2.py3-none-any.whl ../input/lama-whl/pyquery-1.4.3-py3-none-any.whl ../input/lama-whl/chuanconggao-html2json-0.2.4.1-0-g99d7fbb/chuanconggao-html2json-99d7fbb/ ../input/lama-whl/json2html-1.3.0/json2html-1.3.0 ../input/lama-whl/lightgbm-2.3.1-py2.py3-none-manylinux1_x86_64.whl ../input/lama-whl/AutoWoE-1.2.1-py3-none-any.whl ../input/lama-whl/LightAutoML-0.2.14-py3-none-any.whl

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
from shutil import copyfile
import os
from sklearn.metrics import mean_squared_error
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task

In [ ]:
model_dir_path = './models'
saved_model_path = '../input/lama-bert-starter/models/'

if not os.path.exists(model_dir_path):
    os.makedirs(model_dir_path)

for file in os.listdir(saved_model_path):
    src = os.path.join(saved_model_path, file)
    dst = os.path.join(model_dir_path, file)
    copyfile(src, dst)

In [ ]:
def rmse(x, y): return np.sqrt(mean_squared_error(x, y))
with open('../input/lama-bert-starter/LAMA_model.pkl', 'rb') as f:
    automl = pickle.load(f)

In [ ]:
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
def preprocess_text(df):
    df['len_tokens'] = df['excerpt'].str.strip().str.split(' ').apply(len)
    df['len'] = df['excerpt'].str.strip().apply(len)
    df['len_sent'] = df['excerpt'].str.strip().str.split('.').apply(len)
    df['n_comm'] = df['excerpt'].str.strip().str.split(',').apply(len)
    _t = df['excerpt'].str.strip().str.split(' ').values
    df['d_mean'] = [np.sum([j.isdigit() for j in i]) for i in _t]
    df['u_mean'] = [np.sum([j.isupper() for j in i]) for i in _t]
    
preprocess_text(test)

In [ ]:
preds_test = automl.predict(test).data[:, 0]
submission = pd.DataFrame({'id': test.id, 'target': preds_test})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()